In [1]:
import requests
from fake_useragent import UserAgent
import json
from tqdm import tqdm
from time import sleep
# fake useragent
ua = UserAgent()

user_agent = ua.random


# add headers 模拟浏览器正常访问请求
headers = {
    "User-Agent": user_agent,
    "Cookie": r"ttwid=1%7CmyHgF6b1xI5ibG3ITgfcq5ZVvIBbT6DE4VNW2h-JS_I%7C1721306154%7Ccead4d9da5b5fe6ae95fd319f1528c3b7199bb03307759ed95e0e3ce3812be28; tt_webid=7392953585342842430; tt_web_version=new; s_v_web_id=verify_lyr98o17_Df8Qunzk_DC92_4Rhh_B5MA_ptTG9CSea0WY; city_name=%E9%87%8D%E5%BA%86; msToken=-A6plihpMHPZ8KdBbgtAgwfzXN7BC68xK4EoeslAj10vvTlg-P6bDp7jDLu92wErp-fezviZiY9-45b_zD6QaTvYmWoMiFRZHXhPgXpl; rit_city=%E5%85%A8%E5%9B%BD; is_dev=false; is_boe=false; Hm_lvt_3e79ab9e4da287b5752d8048743b95e6=1721309659,1721830218; HMACCOUNT=8160C1392AF61C82; _gid=GA1.2.24302535.1721830225; _gat_gtag_UA_138671306_1=1; _ga=GA1.1.427450835.1721306153; _ga_YB3EWSDTGF=GS1.1.1721830224.8.1.1721830244.40.0.0; Hm_lpvt_3e79ab9e4da287b5752d8048743b95e6=1721830245",
}
# add data

data = [{
        "sh_city_name": "全国"
         , "page": int("{}".format(i))
         , "limit": 20
         } for i in range(1, 500 + 1, 1)]

# net
href = (
    r"https://www.dongchedi.com/motor/pc/sh/sh_sku_list?aid=1839&app_name=auto_web_pc"
)



# response
for _ in tqdm(range(len(data))):

    res = requests.post(href, headers=headers, data=data[_])

    # return status code
    res.raise_for_status

    # judge content encoding
    res.encoding = res.apparent_encoding
    # print(res.json())
    # sleep(0.5)
    with open ('./json/cars{}.json'.format(_+1), 'w', encoding = "utf-8") as f:
        json.dump(res.json(), f)


 90%|████████▉ | 449/500 [06:57<01:23,  1.64s/it]

In [1]:
import os
# find all json files
json_files = []
for root, dirs, files in os.walk('.\json'):
    for file in files:
        if file.endswith('.json'):
            json_files.append(os.path.join(root, file))
json_files[:5]




['.\\json\\cars1.json',
 '.\\json\\cars10.json',
 '.\\json\\cars100.json',
 '.\\json\\cars101.json',
 '.\\json\\cars102.json']

In [2]:
import os
from fontTools.ttLib import TTFont
from PIL import Image, ImageDraw, ImageFont

# 加载字体文件
font_path = './SourceHanSansSCBold.ttf'
font = TTFont(font_path)

# 创建输出文件夹
output_folder = './imgs'
os.makedirs(output_folder, exist_ok=True)

# 获取 cmap 表
cmap = font['cmap'].getBestCmap()

# 提取字形并保存为图像
for codepoint, glyph_name in cmap.items():
    # 创建一个新的图像
    image_size = (200, 200)
    image = Image.new('L', image_size, color=255)
    draw = ImageDraw.Draw(image)

    # 创建字体对象
    pil_font = ImageFont.truetype(font_path, size=150)

    # 获取字符
    char = chr(codepoint)

    # 绘制字符到图像
    draw.text((10, 10), char, font=pil_font, fill=0)

    # 保存图像
    image.save(os.path.join(output_folder, f'{codepoint:04x}.png'))



In [3]:
## 不需要运行
# # 由于数据集部分数据乱码 我们可以读取ttf文件，使用ocr工具识别图片，图片由ttf文件切割而来，组成wordMap

# import os
# import re
# from tqdm import tqdm
# from cnocr import CnOcr



# img_files = []
# for root, dirs, files in os.walk(r'./imgs/'):
#     for file in files:
#         if file.endswith('.png'):
#             img_files.append(os.path.join(root, file))
# # print(img_files[:5])

# img_names = [r'\u{}'.format(re.findall(".*imgs/(.*?)[.]png", img)[0]) for img in img_files]
# print(img_names[:5])

# ocr = CnOcr()


# result = [ocr.ocr_for_single_line(img_file)['text'] for img_file in tqdm(img_files)]
# wordM = dict(zip(img_names, result))

# # def replace_(col):
# #     for key, value in wordM.items():
# #         col = re.sub(key, value, col)
# #     return col

# # print(replace_('\ue463\ue54c.\ue45d\ue4e3\ue40a'))

# # print(replace_('\ue463.\ue534\ue411\ue40a'))
# from pprint import pprint
# pprint(wordM)


['\\ue3e8', '\\ue3e9', '\\ue3ea', '\\ue3eb', '\\ue3ec']


100%|██████████| 362/362 [00:01<00:00, 206.90it/s]


{'\\ue3e8': '候',
 '\\ue3e9': 'p',
 '\\ue3ea': '应',
 '\\ue3eb': '7',
 '\\ue3ec': '夕',
 '\\ue3ed': '学',
 '\\ue3ee': '在',
 '\\ue3f0': 'x',
 '\\ue3f1': '着',
 '\\ue3f2': '原',
 '\\ue3f3': 'B',
 '\\ue3f4': '几',
 '\\ue3f5': '相',
 '\\ue3f6': 'y',
 '\\ue3f7': '处',
 '\\ue3f9': '进',
 '\\ue3fa': '下',
 '\\ue3fb': '长',
 '\\ue3fc': '手',
 '\\ue3fd': 'w',
 '\\ue3fe': '花',
 '\\ue3ff': '后',
 '\\ue400': '1',
 '\\ue401': '发',
 '\\ue402': '写',
 '\\ue403': '别',
 '\\ue404': '期',
 '\\ue405': '声',
 '\\ue406': '还',
 '\\ue407': '友',
 '\\ue408': 'G',
 '\\ue409': '些',
 '\\ue40a': '万',
 '\\ue40b': 'v',
 '\\ue40c': '已',
 '\\ue40d': '路',
 '\\ue40e': '等',
 '\\ue40f': '爱',
 '\\ue410': '每',
 '\\ue411': '5',
 '\\ue412': '有',
 '\\ue413': '结',
 '\\ue414': '回',
 '\\ue415': '音',
 '\\ue416': '像',
 '\\ue417': '其斯',
 '\\ue418': '比',
 '\\ue419': '士',
 '\\ue41a': '现',
 '\\ue41b': '其',
 '\\ue41c': '难',
 '\\ue41d': '4',
 '\\ue41e': '过',
 '\\ue41f': '物',
 '\\ue420': '儿',
 '\\ue421': '名',
 '\\ue422': '打',
 '\\ue423': '都',
 '\\ue424': '

In [2]:
# 按照上个单元格的wordM 替换对应unicode编码（在原始json文件里）
import json
import re
wordMap = {}
with open('wordM.json', 'r') as file:
    wordMap = json.load(file)

# 1. 读取JSON文件
for jsonPath in json_files:
    with open(jsonPath, 'r') as file:
        content = json.load(file)

    # 2. 将字典转换为字符串
    json_str = json.dumps(content)

    # 3. 使用re.sub按键值对进行替换
    for key, value in wordMap.items():
        # 注意：使用re.escape来处理特殊字符
        json_str = re.sub(re.escape(key), value, json_str)

    # 4. 将字符串转换回字典
    updated_data = json.loads(json_str)

    # 5. 将更新后的字典保存回JSON文件
    with open(jsonPath, 'w') as file:
        json.dump(updated_data, file, indent=4)

print("替换完成，JSON文件已更新。")

替换完成，JSON文件已更新。


In [3]:
# read json file from local
import json
import pandas as pd
import os

data = []
# find all json files

for jsonPath in json_files:
    with open(jsonPath, 'r', encoding='utf-8') as f_:

        car_data = json.load(f_)

        # header names
        Headers = [
            "brand_name"
            , "car_name"
            , "car_source_city_name" # 卖车人的城市
            , "sub_title" # 包含 car_year 和 car_mileage
            , "official_price" #官方价格
            , "sh_price"  # 出售价格
            , 'transfer_cnt' #过户次数
            ]

        data_searched = car_data['data']['search_sh_sku_info_list']

        # memory data

        # print(cars)

        for diclst in data_searched:

            # print(replace_(diclst['sh_price']))

            #还可以使用列表生成式
            # dit = [diclst[col]  if col not in 'official_price and sh_price' else replace_(diclst[col]) for col in Headers]
            dit = {col: diclst[col] for col in Headers}
            # print(dit)
            data.append(dit)

        cars = pd.DataFrame(columns=Headers, data=data)




In [4]:
cars

,brand_name,car_name,car_source_city_name,sub_title,official_price,sh_price,transfer_cnt
0,比亚迪,秦EV300 尊享型,沈阳,2017年 | 6.80万公里,24.59万,3.68万,3
1,宝马,X5 M,北京,2011年 | 6.72万公里,201.80万,15.98万,4
2,众泰,2.0T 自动极致之心版,广州,2018年 | 3.90万公里,15.18万,3.18万,0
3,雷克萨斯,300e 纯·悦,南昌,2021年 | 3.24万公里,36.20万,11.98万,0
4,雪佛兰,RS 330T 自动畅快版,杭州,2019年 | 4.34万公里,11.49万,3.88万,0
...,...,...,...,...,...,...,...
9993,路虎,249PS R-Dynamic S 性能版 5座,上海,2021年 | 3.06万公里,38.28万,16.98万,0
9994,宝马,525Li 豪华设计套装,上海,2014年 | 9.20万公里,49.96万,11.89万,0
9995,比亚迪,1.5T 自动智联精英型 7座,郑州,2018年 | 3.25万公里,10.99万,5.33万,1
9996,大众,驰,厦门,2020年 | 4.43万公里,14.77万,7.18万,2


In [5]:
# 分割字段 subtitle
split_df = cars['sub_title'].str.split('|', expand=True)

# 将分割后的 DataFrame 列分别赋值给新的字段
cars['car_year'] = split_df[0]
cars['car_mileage'] = split_df[1]
# 去重前注意先排序
cars = cars.sort_values('brand_name', ignore_index=True)

cars_duplicated = cars.drop_duplicates()
cars_duplicated.drop(columns="sub_title", inplace=True)
cars_duplicated
cars_duplicated.to_csv('./resource/used_cars.csv', index=False, header=True)



<ipython-input-5-70de19e4b059>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cars_duplicated.drop(columns="sub_title", inplace=True)


In [ ]:
## 数据集仍然有部分乱码，``需要删除`` 好像没有了。

